[21 卷积层里的多输入多输出通道](https://www.bilibili.com/video/BV1MB4y1F7of?spm_id_from=333.999.0.0)

### 单输出通道模式
- 每个通道有自己的卷积核，卷积结果为每个卷积核卷积后相加
<img src="picture\屏幕截图 2022-05-04 234542.png"></a>

### 多输出通道模式
- 所谓多通道输出，其实可以理解为用不同的卷积核分别进行单通道输出，再叠加即可
- c:通道数 h w为图像高和宽
- 卷积核也相应的变成了三维
<img src="picture\屏幕截图 2022-05-04 235112.png"></a>

### 二维卷积层
- 输入：通道数 X 图片高 X 图片宽
- 核: 输出通道数 X 输入通道数 X 图片高 X 图片宽
- 偏差：输出通道数 X 输入通道数
- 输出Y: 输出通道数 X 卷积高 X 卷积宽
<img src="picture\屏幕截图 2022-05-05 005648.png"></a>

In [1]:
import torch
from d2l import torch as d2l

# 单输入通道的代码实现
def corr2d_multi_in(X,K):
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [3]:
a = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = torch.tensor([[11, 22, 33], [44, 55, 66], [77, 88, 99]])
c = torch.stack([a, b], 0)
print(a)
print(b)
print(c)

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])
tensor([[11, 22, 33],
        [44, 55, 66],
        [77, 88, 99]])
tensor([[[ 1,  2,  3],
         [ 4,  5,  6],
         [ 7,  8,  9]],

        [[11, 22, 33],
         [44, 55, 66],
         [77, 88, 99]]])


In [4]:
# 多输出通道的代码实现
def corr2d_multi_in_out(X, K):
# 迭代“K”的第0个维度，每次都对输⼊“X”执⾏互相关运算。
# 最后将所有结果都叠加在⼀起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
K = torch.stack((K, K + 1, K + 2), 0)
# print(K)
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [5]:
# 证明1*1卷积等价于全连接
# 这里生成一个全连接层
def corr2d_multi_in_out_1x1(X, K):
    # c_i,c_o 为输入输出通道数
    c_i, h, w = X.shape
    c_o = K.shape[0]
    # 这里是全连接层将矩阵进行展平操作，每个通道都进行展开
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [6]:
# X,K为正态分布的向量
X = torch.normal(0, 1, (3, 3, 3)) 
# 这里K为卷积核，这里显然使用了两个不同卷积核作为输出
K = torch.normal(0, 1, (2, 3, 1, 1))
X,K

(tensor([[[-2.2823, -0.8331, -0.8059],
          [-0.7143, -0.5921,  1.0605],
          [-1.2789, -0.5802,  0.0334]],
 
         [[ 0.2139,  0.3961, -0.1117],
          [ 0.6905, -1.4285, -0.3292],
          [ 2.1695,  0.3954, -1.7959]],
 
         [[ 2.0091,  1.7758, -0.0570],
          [ 1.1330,  0.6528,  1.0920],
          [-0.9390,  1.3047, -0.2772]]]),
 tensor([[[[ 0.7333]],
 
          [[-0.9500]],
 
          [[-0.4692]]],
 
 
         [[[ 0.5505]],
 
          [[ 0.2558]],
 
          [[-0.1276]]]]))

In [7]:
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
print(Y1)
print(Y2)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

tensor([[[-2.8195, -1.8205, -0.4582],
         [-1.7115,  0.6166,  0.5781],
         [-2.5583, -1.4133,  1.8607]],

        [[-1.4581, -0.5839, -0.4650],
         [-0.3612, -0.7747,  0.3602],
         [-0.0292, -0.3848, -0.4056]]])
tensor([[[-2.8195, -1.8205, -0.4582],
         [-1.7115,  0.6166,  0.5781],
         [-2.5583, -1.4133,  1.8607]],

        [[-1.4581, -0.5839, -0.4650],
         [-0.3612, -0.7747,  0.3602],
         [-0.0292, -0.3848, -0.4056]]])


### Q/A
- Q22:每个通道的卷积核的值是不一样的，但是为了计算方便，卷积核大小是一样的
- Q24:卷积核也是通过学习给出的
- Q27:神经网络并不需要对数据进行过多的拆分（例如将数据分为高低频）
- Q29:可以将每个通道分别用卷积核进行卷积，在利用一个(1,1,n)卷积核进行全连接，这么做的好处是计算量大幅度降低（使得手机计算神经网络成为可能），但相应的计算效果会差很多